In [3]:
"""
去youtube采集下载项
去解析的网站把视频解析出来
最后下载

"""

'\n去youtube采集下载项\n去解析的网站把视频解析出来\n最后下载\n\n'

In [4]:
# -*- coding: utf-8 -*-
import requests
import re
import json
from urllib import parse
import time

proxies = {
    "https": "socks5://127.0.0.1:26583",
}

In [5]:
def get_down_load_url():
    url_list = []
    s = requests.Session()

    headers = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'
    }
    url = 'https://www.youtube.com/channel/UCiLtwQM-6PbImq5QCYALumg/videos'
    resp = s.get(url, headers=headers, proxies=proxies)
    pattern = re.compile(r'{"responseContext":{"serviceTrackingParams":.*?}}};')
    pattern_token = re.compile(r'"ID_TOKEN":(.*?),')
    #非贪婪匹配：.*?
    target_json = re.findall(pattern, resp.text)[0][:-1]
    id_token = re.findall(pattern_token, resp.text)
    print(id_token)
    target_json = json.loads(target_json)
    target_json = target_json['contents']['twoColumnBrowseResultsRenderer']['tabs'][1]['tabRenderer']['content']['sectionListRenderer']['contents'][0]['itemSectionRenderer']['contents'][0]['gridRenderer']
    items_ = target_json['items']
    for item_ in items_:
        print(item_['gridVideoRenderer']['videoId'])
        url_list.append(item_['gridVideoRenderer']['videoId'])
    next_msg = target_json['continuations'][0]['nextContinuationData']
    next_continuation = parse.quote(next_msg['continuation'])
    next_itct = next_msg['clickTrackingParams']
    # print(len(items_))
    print(next_continuation)
    print(next_itct)
    json_url = 'https://www.youtube.com/browse_ajax?ctoken=%s&continuation=%s&itct=%s' % (next_continuation, next_continuation, next_itct)
    #这里是抓取的页数。。。改为10 就抓10页的视频。
    for x in range(3):
        headers_json = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
            'X-YouTube-Variants-Checksum':'',
            'X-YouTube-Client-Name': '1',
            'X-YouTube-Client-Version': '2.20190730.03.00'
        }
        resp = s.get(json_url, headers=headers_json, proxies=proxies)
        pattern_next = re.compile('{"nextContinuationData":{"continuation":"(.*?)","clickTrackingParams":"(.*?)"}}')
        next_msg = re.findall(pattern_next, resp.text)[0]
        next_continuation = next_msg[0]
        next_itct = next_msg[1]
        json_url = 'https://www.youtube.com/browse_ajax?ctoken=%s&continuation=%s&itct=%s' % (next_continuation, next_continuation, next_itct)
        pattern_items = re.compile(r'watchEndpoint":{"videoId":"(.*?)"}}')
        items_ = re.findall(pattern_items, resp.text)
        for item_ in items_:
            print(item_)
            url_list.append(item_)
        print(next_continuation)
        print(next_itct)
    return url_list

In [6]:
def get_down_url(url):
    s = requests.Session()
    post_url = 'https://www.clipconverter.cc/check.php'
    post_data = {
        'mediaurl': url,
        'filename': '',
        'filetype': '',
        'format': '',
        'audiovol': '0',
        'audiochannel': '2',
        'audiobr': '128',
        'videobr': '224',
        'videores': '352x288',
        'videoaspect': '',
        'customres': '320x240',
        'timefrom-start': '1',
        'timeto-end': '1',
        'id3-artist': '',
        'id3-title': '',
        'id3-album': 'ClipConverter.cc',
        'auto': '0',
        'hash': '',
        'image': '',
        'org-filename': '',
        'videoid': '',
        'pattern': '',
        'server': '',
        'serverinterface': '',
        'service': '',
        'ref': '',
        'lang': 'en',
        'client_urlmap': 'none',
        'ipv6': 'false',
        'addon_urlmap': '',
        'cookie': '',
        'addon_cookie': '',
        'addon_title': '',
        'ablock': '1',
        'clientside': '0',
        'addon_page': 'none',
        'verify': '',
        'result': '',
        'again': '',
        'addon_browser': '',
        'addon_version': '',
    }
    post_headsers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36'\
                      ' (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    }
    resp = s.post(post_url, headers=post_headsers, data= post_data, proxies=proxies)
    resp = json.loads(resp.text)
    msg_1080 = resp['url'][0]
    # print(msg_1080)
    file_name = resp['filename']
    post_data = {
        'mediaurl': url,
        'url': msg_1080['url'],
        'filename': resp['filename'],
        'filetype': 'MP4',
        'format': '',
        'audiovol': '0',
        'audiochannel': '2',
        'audiobr': '128',
        'videobr': '224',
        'videores': '352x288',
        'videoaspect': '',
        'customres': '320x240',
        'timefrom-start': '1',
        'timeto-end': '1',
        'id3-artist': resp['id3artist'],
        'id3-title': resp['id3title'],
        'id3-album': 'ClipConverter.cc',
        'auto': '0',
        'hash': '',
        'image': resp['thumb'],
        'org-filename': resp['filename'],
        'videoid': resp['videoid'],
        'pattern': resp['pattern'],
        'server': resp['server'],
        'serverinterface': resp['serverinterface'],
        'service': 'YouTube',
        'ref': '',
        'lang': 'en',
        'client_urlmap': 'none',
        'ipv6': 'false',
        'addon_urlmap': '',
        'cookie': '',
        'addon_cookie': '',
        'addon_title': '',
        'ablock': '1',
        'clientside': '0',
        'addon_page': 'none',
        'verify': resp['verify'],
        'result': '',
        'again': '',
        'addon_browser': '',
        'addon_version': '',
    }
    resp = s.post(post_url, headers=post_headsers, data=post_data, proxies=proxies)
    # print(resp.text)
    # print(post_data)
    hash = json.loads(resp.text)['hash']
    parse_url = 'https://www.clipconverter.cc/convert/%s/?ajax' % hash
    resp = s.get(parse_url, headers=post_headsers, proxies=proxies)
    # print(resp.text)
    pattern_status_url = re.compile(r'statusurl = "(.*?)";')
    status_url = re.findall(pattern_status_url, resp.text)[0]
    while(1):
        # https://srv73.clipconverter.cc/api/?id=55PykyEs&json&key=l5iZbHJooWdmZbV70szTe6ie3pemp56q4piYiLKb4Xen&callback=jsonp1564903062741
        now = int(time.time())
        status_url = status_url + '&callback=jsonp' + str(now)
        resp = s.get(status_url).text
        pattern_ = '{"@attributes":.*}'
        resp = re.findall(pattern_, resp)[0]
        resp = json.loads(resp)
        # print(resp)
        if 'finished' in json.dumps(resp['status']):
            down_url = resp['downloadurl']
            print(down_url)
            #如果下载慢，就把url.txt的链接复制到迅雷下载，注释掉下边3句
            # content = s.get(down_url, headers=post_headsers)
            # with open(file_name + '\.mp4', 'ab+') as f:
            #     f.write(content.content)
            with open('url.txt', 'a+') as f:
                f.write(down_url + '\n')
            break
        else:
            try:
                status_ = resp['status']
                per_cent = status_['@attributes']['percent']
                info = status_['@attributes']['info']
                print('%s 已经录制 %%%s ,进度： %s' % (file_name, per_cent, info))
                time.sleep(5)
            except Exception as e:
                time.sleep(5)
                continue
    print('%s 文件下载完毕！' % file_name)

In [7]:
if __name__ == '__main__':

    url_list = get_down_load_url()
    for url in url_list:
        url = 'https://www.youtube.com/watch?v=%s' % url
        print(url)
        get_down_url(url)
    # print(url_list)


['null']
9XquALnmkzA
0o77IRNn2FE
5YCX8ZWFixo
Qyg8uQO4WUI
IjpNZzmOZoU
EpUINnf7m-Y
_9KI1_yYYiM
4PVBp4wNKF0
-Fwtaoe7ukc
5mqnxdu6iAE
8Os3UWkjDtk
uUudaXogwLQ
bNEZV0l0M7I
uS5GaVW0d18
L0cF768PWqo
T0AIGTU4YHA
73LcykPFQ9c
Ix025Y1zgl8
LcTBIZfAwmo
66jXj28s6c4
fDyPByIaWEw
0mG84rnEL8c
rNyK41l_rNY
Iz9XGcULsG8
gnQnsCEMlqU
z77HebA8Z8M
tMRGUnfXHvM
7PLheBk54i4
TmM7wxIfhCc
L_W0C15WLu0
4qmFsgI0EhhVQ2lMdHdRTS02UGJJbXE1UUNZQUx1bWcaGEVnWjJhV1JsYjNNZ0FEZ0JlZ0V5dUFFQQ%253D%253D
CDkQybcCIhMI0J_D0bTr4wIVhT8qCh213w3a
GkcyzvVsZ78
sPIMtZzONAg
btkMbCiGj3E
_7D92tVdhPA
A-DOPGWQzHI
Msld1T3S7ek
sWXwsMIHJJo
cDH-2tytLoI
SvL33TVf168
2jntOUuNFQY
4xlQ3v5ipKo
sYMcMWFBfK0
O7diMgrmdDk
PyJhsDp6CK8
KP4vacded7E
igDfJUoAy0c
KmdPBoEcrx4
-g3TDzqC9wU
Yi9_iU0huZg
v_JQ-U7lH70
_tzC58ZMI8Y
84KEfpiPBRs
1FDfvc2sIyM
yobomjA4ELc
decqE0Ii4n0
oOJL-ji5rFM
GX26mhAMbpg
B3Ix-iZn11A
GCCxa50cIDw
GIRjpCcXH4A
4qmFsgI0EhhVQ2lMdHdRTS02UGJJbXE1UUNZQUx1bWcaGEVnWjJhV1JsYjNNZ0FEZ0JlZ0V6dUFFQQ%3D%3D
CAIQybcCIhMIt5aH0rTr4wIViTcqCh3x8QUI
GIRjpCcXH4A
RXYF61W3zic

【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %0 ,进度： 16 KB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %1 ,进度： 20.27 MB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %3 ,进度： 58 MB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %5 ,进度： 78.62 MB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %6 ,进度： 98.39 MB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %7 ,进度： 120 MB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %9 ,进度： 140.62 MB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %10 ,进度： 164.58 MB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %11 ,进度： 182.15 MB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %13 ,进度： 202.67 MB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %14 ,进度： 223.22 MB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %15 ,进度： 243.76 MB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %17 ,进度： 264.64 MB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %18 ,进度： 285.26 MB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %20 ,进度： 305.87 MB / 1.49 GB (Video)
【LPL夏季賽】第9週 IG v

【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %65 ,进度： 30 MB / 45.75 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %66 ,进度： 30.47 MB / 45.75 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %67 ,进度： 30.94 MB / 45.75 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %69 ,进度： 31.87 MB / 45.75 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %70 ,进度： 32.41 MB / 45.75 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %71 ,进度： 32.87 MB / 45.75 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %72 ,进度： 33.34 MB / 45.75 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %73 ,进度： 33.84 MB / 45.75 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %0 ,进度： 
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %0 ,进度： 
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %0 ,进度： 
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %13 ,进度： 06:52 / 49:24
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %32 ,进度： 15:56 / 49:24
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %45 ,进度： 22:18 / 49:24
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %62 ,进度： 30:54 / 49:24
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %73 ,进度： 36:23 / 49:24
【LPL夏季賽】第9週 IG vs BLG #3 已经录制 %91 ,进度： 45:09 / 49:24
http://srv78.clipconverter.cc/do

【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %26 ,进度： 12.69 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %27 ,进度： 13.17 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %28 ,进度： 13.66 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %30 ,进度： 14.59 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %31 ,进度： 15.11 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %32 ,进度： 15.61 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %33 ,进度： 16.11 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %34 ,进度： 16.62 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %36 ,进度： 17.56 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %37 ,进度： 18.03 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %38 ,进度： 18.5 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %39 ,进度： 18.97 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %41 ,进度： 19.91 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %42 ,进度： 20.37 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週 IG vs BLG #2 已经录制 %43 ,进度： 20.89 MB / 47.8 MB (Audio)
【LPL夏季賽】第9週

【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %13 ,进度： 5.39 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %14 ,进度： 5.86 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %17 ,进度： 6.8 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %18 ,进度： 7.31 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %19 ,进度： 7.78 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %20 ,进度： 8.25 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %24 ,进度： 9.72 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %27 ,进度： 10.7 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %28 ,进度： 11.22 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %29 ,进度： 11.69 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %30 ,进度： 12.16 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %33 ,进度： 13.08 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %34 ,进度： 13.59 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %35 ,进度： 14.12 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs SB #2 已经录制 %36 ,进度： 14.62 MB / 39.63 MB (Audio)
【LCK夏季賽】第8週 KZ vs S

SSLError: HTTPSConnectionPool(host='srv73.clipconverter.cc', port=443): Max retries exceeded with url: /api/?id=4Er7MoQz&json&key=l5iZbWlonWlqbLV70szTe6ie3pemp56q4peoqnB914Ou&callback=jsonp1564999331&callback=jsonp1564999338&callback=jsonp1564999343&callback=jsonp1564999348&callback=jsonp1564999354&callback=jsonp1564999359&callback=jsonp1564999364&callback=jsonp1564999370&callback=jsonp1564999375&callback=jsonp1564999380&callback=jsonp1564999386&callback=jsonp1564999391&callback=jsonp1564999396&callback=jsonp1564999402&callback=jsonp1564999407&callback=jsonp1564999413&callback=jsonp1564999418&callback=jsonp1564999424&callback=jsonp1564999429&callback=jsonp1564999434&callback=jsonp1564999440&callback=jsonp1564999446&callback=jsonp1564999451&callback=jsonp1564999456&callback=jsonp1564999462&callback=jsonp1564999467&callback=jsonp1564999472&callback=jsonp1564999478&callback=jsonp1564999483&callback=jsonp1564999488&callback=jsonp1564999495&callback=jsonp1564999500&callback=jsonp1564999506&